# 🎄 Santa 2025 - Ultra-Optimized Tree Packing

**Target: Score < 50**

Key strategies:
- Aggressive simulated annealing with many iterations
- Rotation optimization for interlocking trees
- Multi-phase optimization pipeline
- Incremental building with local refinement

In [ ]:
import numpy as np
import pandas as pd
import os
import time
from typing import List, Tuple, Optional

try:
    from tqdm.auto import tqdm
except ImportError:
    !pip install tqdm -q
    from tqdm.auto import tqdm

try:
    from numba import njit
except ImportError:
    !pip install numba -q
    from numba import njit

np.random.seed(42)
print(f"NumPy: {np.__version__}")

In [ ]:
# ============================================================
# TREE POLYGON
# ============================================================

TREE_POLYGON = np.array([
    [0.0, 1.0], [-0.25, 0.6], [-0.15, 0.6], [-0.35, 0.3], [-0.20, 0.3],
    [-0.45, 0.0], [-0.10, 0.0], [-0.10, -0.20], [0.10, -0.20], [0.10, 0.0],
    [0.45, 0.0], [0.20, 0.3], [0.35, 0.3], [0.15, 0.6], [0.25, 0.6],
], dtype=np.float64)

# Try to load from Kaggle
for path in ['/kaggle/input/santa-2025', '/kaggle/input/santa-2025-christmas-tree-packing-challenge', '.']:
    tree_file = os.path.join(path, 'tree.csv')
    if os.path.exists(tree_file):
        df = pd.read_csv(tree_file)
        if 'x' in df.columns and 'y' in df.columns:
            TREE_POLYGON = df[['x', 'y']].values.astype(np.float64)
            print(f"Loaded tree from {tree_file}")
            break

NUM_VERTICES = len(TREE_POLYGON)
TREE_WIDTH = TREE_POLYGON[:, 0].max() - TREE_POLYGON[:, 0].min()
TREE_HEIGHT = TREE_POLYGON[:, 1].max() - TREE_POLYGON[:, 1].min()
print(f"Tree: {NUM_VERTICES} vertices, {TREE_WIDTH:.3f} x {TREE_HEIGHT:.3f}")

In [ ]:
# ============================================================
# NUMBA GEOMETRY
# ============================================================

@njit(cache=True)
def rotate_poly(poly, deg):
    rad = np.radians(deg)
    c, s = np.cos(rad), np.sin(rad)
    result = np.empty_like(poly)
    for i in range(len(poly)):
        result[i, 0] = poly[i, 0] * c - poly[i, 1] * s
        result[i, 1] = poly[i, 0] * s + poly[i, 1] * c
    return result

@njit(cache=True)
def transform_poly(poly, x, y, deg):
    rad = np.radians(deg)
    c, s = np.cos(rad), np.sin(rad)
    result = np.empty_like(poly)
    for i in range(len(poly)):
        result[i, 0] = poly[i, 0] * c - poly[i, 1] * s + x
        result[i, 1] = poly[i, 0] * s + poly[i, 1] * c + y
    return result

@njit(cache=True)
def get_bbox(poly):
    min_x = max_x = poly[0, 0]
    min_y = max_y = poly[0, 1]
    for i in range(1, len(poly)):
        if poly[i, 0] < min_x: min_x = poly[i, 0]
        if poly[i, 0] > max_x: max_x = poly[i, 0]
        if poly[i, 1] < min_y: min_y = poly[i, 1]
        if poly[i, 1] > max_y: max_y = poly[i, 1]
    return min_x, min_y, max_x, max_y

@njit(cache=True)
def bbox_overlaps(b1, b2, eps=0.0):
    return (b1[0] < b2[2] - eps and b1[2] > b2[0] + eps and
            b1[1] < b2[3] - eps and b1[3] > b2[1] + eps)

print("Geometry compiled.")

In [ ]:
# ============================================================
# SAT COLLISION (Optimized)
# ============================================================

@njit(cache=True)
def sat_overlap(poly1, poly2, eps=1e-9):
    """SAT collision test."""
    for p in range(2):
        poly = poly1 if p == 0 else poly2
        n = len(poly)
        for i in range(n):
            j = (i + 1) % n
            nx = -(poly[j, 1] - poly[i, 1])
            ny = poly[j, 0] - poly[i, 0]
            length = np.sqrt(nx*nx + ny*ny)
            if length < 1e-10:
                continue
            nx /= length
            ny /= length
            
            min1 = max1 = poly1[0, 0] * nx + poly1[0, 1] * ny
            for k in range(1, len(poly1)):
                proj = poly1[k, 0] * nx + poly1[k, 1] * ny
                if proj < min1: min1 = proj
                if proj > max1: max1 = proj
            
            min2 = max2 = poly2[0, 0] * nx + poly2[0, 1] * ny
            for k in range(1, len(poly2)):
                proj = poly2[k, 0] * nx + poly2[k, 1] * ny
                if proj < min2: min2 = proj
                if proj > max2: max2 = proj
            
            if max1 <= min2 + eps or max2 <= min1 + eps:
                return False
    return True

@njit(cache=True)
def collides(base, x1, y1, d1, x2, y2, d2, eps=1e-9):
    p1 = transform_poly(base, x1, y1, d1)
    p2 = transform_poly(base, x2, y2, d2)
    b1, b2 = get_bbox(p1), get_bbox(p2)
    if not bbox_overlaps(b1, b2, eps):
        return False
    return sat_overlap(p1, p2, eps)

@njit(cache=True)
def any_collision(base, pl, eps=1e-9):
    n = len(pl)
    for i in range(n):
        for j in range(i + 1, n):
            if collides(base, pl[i,0], pl[i,1], pl[i,2], pl[j,0], pl[j,1], pl[j,2], eps):
                return True
    return False

@njit(cache=True)
def count_collisions_idx(base, pl, idx, eps=1e-9):
    cnt = 0
    for j in range(len(pl)):
        if j != idx:
            if collides(base, pl[idx,0], pl[idx,1], pl[idx,2], pl[j,0], pl[j,1], pl[j,2], eps):
                cnt += 1
    return cnt

print("Collision detection compiled.")

In [ ]:
# ============================================================
# BOUNDING BOX
# ============================================================

@njit(cache=True)
def compute_side(base, pl):
    if len(pl) == 0:
        return 0.0
    minx = miny = 1e9
    maxx = maxy = -1e9
    for i in range(len(pl)):
        poly = transform_poly(base, pl[i,0], pl[i,1], pl[i,2])
        b = get_bbox(poly)
        if b[0] < minx: minx = b[0]
        if b[1] < miny: miny = b[1]
        if b[2] > maxx: maxx = b[2]
        if b[3] > maxy: maxy = b[3]
    return max(maxx - minx, maxy - miny)

@njit(cache=True)
def center_pl(base, pl):
    if len(pl) == 0:
        return pl.copy()
    minx = miny = 1e9
    maxx = maxy = -1e9
    for i in range(len(pl)):
        poly = transform_poly(base, pl[i,0], pl[i,1], pl[i,2])
        b = get_bbox(poly)
        if b[0] < minx: minx = b[0]
        if b[1] < miny: miny = b[1]
        if b[2] > maxx: maxx = b[2]
        if b[3] > maxy: maxy = b[3]
    cx, cy = (minx + maxx) / 2, (miny + maxy) / 2
    result = pl.copy()
    for i in range(len(result)):
        result[i, 0] -= cx
        result[i, 1] -= cy
    return result

print("Bounding box compiled.")

In [ ]:
# ============================================================
# SIMULATED ANNEALING (Aggressive)
# ============================================================

@njit(cache=True)
def sa_optimize(base, pl, n_iter, t_init, t_min, eps=1e-9):
    """Aggressive simulated annealing."""
    n = len(pl)
    if n <= 1:
        return pl.copy()
    
    current = pl.copy()
    best = current.copy()
    
    # Score = side^2 + penalty
    def score(p):
        s = compute_side(base, p)
        sc = s * s
        for i in range(len(p)):
            for j in range(i+1, len(p)):
                if collides(base, p[i,0], p[i,1], p[i,2], p[j,0], p[j,1], p[j,2], eps):
                    sc += 100.0
        return sc
    
    cur_score = score(current)
    best_score = cur_score
    
    t = t_init
    cooling = (t_min / t_init) ** (1.0 / n_iter)
    
    for it in range(n_iter):
        idx = np.random.randint(0, n)
        old_x, old_y, old_d = current[idx, 0], current[idx, 1], current[idx, 2]
        
        move = np.random.randint(0, 4)
        if move == 0:  # Small translation
            current[idx, 0] += (np.random.random() - 0.5) * t * 2
            current[idx, 1] += (np.random.random() - 0.5) * t * 2
        elif move == 1:  # Rotation
            current[idx, 2] = (current[idx, 2] + (np.random.random() - 0.5) * t * 180) % 360
        elif move == 2:  # Both
            current[idx, 0] += (np.random.random() - 0.5) * t
            current[idx, 1] += (np.random.random() - 0.5) * t
            current[idx, 2] = (current[idx, 2] + (np.random.random() - 0.5) * t * 90) % 360
        else:  # Swap with another
            j = np.random.randint(0, n)
            if j != idx:
                current[idx, 0], current[j, 0] = current[j, 0], current[idx, 0]
                current[idx, 1], current[j, 1] = current[j, 1], current[idx, 1]
        
        new_score = score(current)
        delta = new_score - cur_score
        
        if delta < 0 or np.random.random() < np.exp(-delta / (t + 0.0001)):
            cur_score = new_score
            if new_score < best_score:
                best = current.copy()
                best_score = new_score
        else:
            current[idx, 0], current[idx, 1], current[idx, 2] = old_x, old_y, old_d
            if move == 3 and j != idx:
                current[idx, 0], current[j, 0] = current[j, 0], current[idx, 0]
                current[idx, 1], current[j, 1] = current[j, 1], current[idx, 1]
        
        t *= cooling
    
    return best

print("SA compiled.")

In [ ]:
# ============================================================
# REPAIR & COMPACT
# ============================================================

@njit(cache=True)
def repair(base, pl, max_iter=500, eps=1e-9):
    result = pl.copy()
    n = len(result)
    for it in range(max_iter):
        has_col = False
        forces = np.zeros((n, 2))
        for i in range(n):
            for j in range(i+1, n):
                if collides(base, result[i,0], result[i,1], result[i,2],
                           result[j,0], result[j,1], result[j,2], eps):
                    has_col = True
                    dx = result[j,0] - result[i,0]
                    dy = result[j,1] - result[i,1]
                    dist = np.sqrt(dx*dx + dy*dy) + 1e-10
                    push = 0.03 * (1 + it/50)
                    forces[i, 0] -= dx/dist * push
                    forces[i, 1] -= dy/dist * push
                    forces[j, 0] += dx/dist * push
                    forces[j, 1] += dy/dist * push
        if not has_col:
            break
        for i in range(n):
            result[i, 0] += forces[i, 0]
            result[i, 1] += forces[i, 1]
    return result

@njit(cache=True)
def compact(base, pl, max_iter=200, eps=1e-9):
    result = pl.copy()
    n = len(result)
    step = 0.015
    for _ in range(max_iter):
        improved = False
        cx = np.mean(result[:, 0])
        cy = np.mean(result[:, 1])
        for i in range(n):
            dx, dy = cx - result[i, 0], cy - result[i, 1]
            dist = np.sqrt(dx*dx + dy*dy)
            if dist < step:
                continue
            old_x, old_y = result[i, 0], result[i, 1]
            result[i, 0] += dx * step / dist
            result[i, 1] += dy * step / dist
            col = False
            for j in range(n):
                if i != j:
                    if collides(base, result[i,0], result[i,1], result[i,2],
                               result[j,0], result[j,1], result[j,2], eps):
                        col = True
                        break
            if col:
                result[i, 0], result[i, 1] = old_x, old_y
            else:
                improved = True
        if not improved:
            break
    return result

@njit(cache=True)
def local_opt_rotations(base, pl, eps=1e-9):
    """Try rotating each tree to find better packing."""
    result = pl.copy()
    n = len(result)
    for i in range(n):
        best_side = compute_side(base, result)
        best_deg = result[i, 2]
        for d in range(0, 360, 15):
            result[i, 2] = float(d)
            # Check collisions
            col = False
            for j in range(n):
                if i != j:
                    if collides(base, result[i,0], result[i,1], result[i,2],
                               result[j,0], result[j,1], result[j,2], eps):
                        col = True
                        break
            if not col:
                s = compute_side(base, result)
                if s < best_side:
                    best_side = s
                    best_deg = float(d)
        result[i, 2] = best_deg
    return result

print("Repair/compact compiled.")

In [ ]:
# ============================================================
# PLACEMENT GENERATORS
# ============================================================

def gen_spiral(n):
    pl = np.zeros((n, 3))
    if n == 0:
        return pl
    pl[0] = [0, 0, 0]
    angle = 0.0
    radius = 0.3
    golden = 137.508 * np.pi / 180
    for i in range(1, n):
        pl[i] = [radius * np.cos(angle), radius * np.sin(angle), (i * 47) % 360]
        angle += golden
        radius += 0.12 / np.sqrt(i + 1)
    return pl

def gen_hex(n):
    pl = np.zeros((n, 3))
    idx = 0
    row = 0
    sx, sy = 0.5, 0.4
    while idx < n:
        offset = (row % 2) * sx * 0.5
        cols = int(np.ceil(np.sqrt(n))) + 1
        for col in range(cols):
            if idx >= n:
                break
            pl[idx] = [col * sx + offset, row * sy, ((row + col) * 60) % 360]
            idx += 1
        row += 1
    cx, cy = np.mean(pl[:, 0]), np.mean(pl[:, 1])
    pl[:, 0] -= cx
    pl[:, 1] -= cy
    return pl

print("Generators ready.")

In [ ]:
# ============================================================
# MAIN SOLVER
# ============================================================

def solve(n, base, prev=None):
    eps = 1e-9
    
    if n == 1:
        pl = np.array([[0.0, 0.0, 0.0]])
        return pl, compute_side(base, pl)
    
    best_pl = None
    best_side = 1e9
    
    # Scale iterations based on n
    base_iter = 10000
    iters = min(base_iter + n * 200, 50000)
    
    # Strategy 1: Extend previous
    if prev is not None and len(prev) == n - 1:
        for att in range(8):
            ext = np.zeros((n, 3))
            ext[:n-1] = prev.copy()
            # Add at different positions
            r = compute_side(base, prev) * 0.5
            a = att * 45 * np.pi / 180
            ext[n-1] = [r * np.cos(a), r * np.sin(a), att * 30]
            
            ext = repair(base, ext, 300, eps)
            ext = sa_optimize(base, ext, iters // 2, 1.5, 0.001, eps)
            ext = repair(base, ext, 200, eps)
            ext = compact(base, ext, 100, eps)
            ext = local_opt_rotations(base, ext, eps)
            
            if not any_collision(base, ext, eps):
                s = compute_side(base, ext)
                if s < best_side:
                    best_side = s
                    best_pl = center_pl(base, ext)
    
    # Strategy 2: Fresh starts
    for gen in [gen_spiral, gen_hex]:
        for temp in [2.0, 1.0]:
            init = gen(n)
            init = repair(base, init, 400, eps)
            opt = sa_optimize(base, init, iters, temp, 0.0001, eps)
            opt = repair(base, opt, 300, eps)
            opt = compact(base, opt, 150, eps)
            opt = local_opt_rotations(base, opt, eps)
            
            if not any_collision(base, opt, eps):
                s = compute_side(base, opt)
                if s < best_side:
                    best_side = s
                    best_pl = center_pl(base, opt)
    
    # Fallback
    if best_pl is None or any_collision(base, best_pl, eps):
        cols = max(1, int(np.ceil(np.sqrt(n))))
        pl = np.zeros((n, 3))
        for i in range(n):
            pl[i] = [(i % cols) * 1.0, (i // cols) * 1.2, 0]
        pl[:, 0] -= np.mean(pl[:, 0])
        pl[:, 1] -= np.mean(pl[:, 1])
        best_pl = pl
        best_side = compute_side(base, best_pl)
    
    return best_pl, best_side

print("Solver ready!")

In [ ]:
# ============================================================
# SOLVE ALL
# ============================================================

MAX_N = 200

print("="*60)
print("🎄 Santa 2025 - Ultra-Optimized Packing")
print("="*60)

solutions = {}
sides = {}
prev = None

start = time.time()

# Warm up
print("Warming up JIT...")
_ = solve(3, TREE_POLYGON, None)
print("Ready!")

pbar = tqdm(range(1, MAX_N + 1), desc="Solving", unit="puzzle")

for n in pbar:
    t0 = time.time()
    pl, side = solve(n, TREE_POLYGON, prev)
    
    # Extra validation
    if any_collision(TREE_POLYGON, pl, 1e-9):
        pl = repair(TREE_POLYGON, pl, 1000, 1e-9)
        side = compute_side(TREE_POLYGON, pl)
    
    solutions[n] = pl
    sides[n] = side
    prev = pl
    
    elapsed = time.time() - t0
    pbar.set_postfix({'n': n, 'side': f'{side:.3f}', 'time': f'{elapsed:.1f}s'})

total_time = time.time() - start
print(f"\n✅ Done in {total_time:.1f}s ({total_time/60:.1f} min)")

In [ ]:
# ============================================================
# VALIDATION
# ============================================================

print("="*60)
print("🔍 Validation")
print("="*60)

failed = []
for n in tqdm(range(1, MAX_N + 1), desc="Validating"):
    if n not in solutions:
        failed.append((n, "Missing"))
    elif len(solutions[n]) != n:
        failed.append((n, f"Count: {len(solutions[n])}"))
    elif any_collision(TREE_POLYGON, solutions[n], 1e-9):
        failed.append((n, "Overlap"))

if failed:
    print(f"\n⚠️ {len(failed)} issues, repairing...")
    for n, _ in tqdm(failed, desc="Repairing"):
        solutions[n] = repair(TREE_POLYGON, solutions[n], 2000, 1e-9)
        if any_collision(TREE_POLYGON, solutions[n], 1e-9):
            solutions[n][:, 0] *= 2
            solutions[n][:, 1] *= 2
            solutions[n] = repair(TREE_POLYGON, solutions[n], 1000, 1e-9)
        sides[n] = compute_side(TREE_POLYGON, solutions[n])
    still = [n for n, _ in failed if any_collision(TREE_POLYGON, solutions[n], 1e-9)]
    if still:
        print(f"❌ Still failed: {still}")
    else:
        print("✅ All fixed!")
else:
    print("\n✅ ALL VALID")

In [ ]:
# ============================================================
# STATISTICS
# ============================================================

print("="*60)
print("📊 Statistics")
print("="*60)

vals = [sides[n] for n in range(1, MAX_N + 1)]
print(f"Side range: {min(vals):.3f} - {max(vals):.3f}")
print(f"Mean side: {np.mean(vals):.3f}")

score = sum(sides[n]**2 / n for n in range(1, MAX_N + 1))
print(f"\n🎯 Estimated Score: {score:.4f}")

print("\nWorst:")
for n, s in sorted(sides.items(), key=lambda x: x[1], reverse=True)[:5]:
    print(f"  n={n}: {s:.3f}")

print("\nBest:")
for n, s in sorted(sides.items(), key=lambda x: x[1])[:5]:
    print(f"  n={n}: {s:.3f}")

In [ ]:
# ============================================================
# CREATE SUBMISSION
# ============================================================

print("Creating submission...")

rows = []
for n in range(1, MAX_N + 1):
    for i, p in enumerate(solutions[n]):
        rows.append({
            'id': f'{n:03d}_{i}',
            'x': f's{p[0]:.6f}',
            'y': f's{p[1]:.6f}',
            'deg': f's{p[2]:.6f}',
        })

submission = pd.DataFrame(rows)
print(f"Shape: {submission.shape}")
print(f"Expected: ({sum(range(1, MAX_N+1))}, 4)")
print(submission.head(10))

In [ ]:
# ============================================================
# SAVE
# ============================================================

valid = all(not any_collision(TREE_POLYGON, solutions[n], 1e-9) for n in range(1, MAX_N + 1))

if valid:
    submission.to_csv('submission.csv', index=False)
    print("✅ submission.csv saved!")
    print(f"Size: {os.path.getsize('submission.csv') / 1024:.1f} KB")
else:
    print("❌ Validation failed")

print("\n" + "="*60)
print("🎄 SUMMARY")
print("="*60)
print(f"Puzzles: {len(solutions)}/{MAX_N}")
print(f"Valid: {valid}")
print(f"Time: {total_time:.1f}s ({total_time/60:.1f} min)")
print(f"Score: {score:.4f}")
print("="*60)